In [1]:
#imports
from bs4 import BeautifulSoup
import requests
import nltk
import csv
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from textblob import TextBlob
from wordcloud import WordCloud
stop_words = set(stopwords.words('english'))

In [2]:
#import reviews
file = open("C:/Users/nedei/Documents/School/NLP/test.csv","r")
csv_reader = csv.reader(file)
urls = []
for row in csv_reader:
    urls.extend(row)
reviews = []
tags = []
for x in urls:
    result = requests.get(x)
    doc = BeautifulSoup(result.text, "html.parser")
    tag = doc.title
    tag = tag.text.strip()
    review = doc.find('div',{'class':'text show-more__control'})
    review = review.text.strip()
    tokens = nltk.word_tokenize(review)
    tokens = [token for token in tokens if token.isalnum()]
    junk = tokens.pop(0)
    words = []
    for x in tokens:
        if x not in stop_words:
            words.append(x)
            review = ' '.join(words)
    reviews.append(review)
    tags.append(tag)


### Perform a vocabulary-based sentiment analysis of the movie reviews you used in homework 5 and homework 7, by doing the following:
#### 1.	In Python, load one of the sentiment vocabularies referenced in the textbook, and run the sentiment analyzer as explained in the corresponding reference. Add words to the sentiment vocabulary, if you think you need to, to better fit your particular text collection.


In [23]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
labels = []
sent = []
def sentiment_scores(review):
    
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(review)
    sent.append(sentiment_dict['compound'])
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        label = "Positive"
        #print("Positive")
 
    elif sentiment_dict['compound'] <= - 0.05 :
        label = "Negative"
        #print("Negative")
 
    else :
        label = "Neutral"
        #print("Neutral")
    labels.append(label)


In [24]:
for x in reviews:
    sentiment_scores(x)

In [25]:
df=pd.DataFrame(list(zip(tags,sent,labels)),columns=['Tag','Sentiment Score','Label'])
df.head()

,Tag,Sentiment Score,Label
0,MartinHafer's Review of Casablanca - IMDb,0.9990,Positive
1,Tweekums's Review of Casablanca - IMDb,0.9764,Positive
2,ma-cortes's Review of Casablanca - IMDb,0.9951,Positive
3,moonspinner55's Review of Casablanca - IMDb,0.9756,Positive
4,TheLittleSongbird's Review of Casablanca - IMDb,0.9964,Positive


#### 2.	For each of the clusters you created in homework 7, compute the average, median, high, and low sentiment scores for each cluster. Explain whether you think this reveals anything interesting about the clusters.

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
vectorizer = TfidfVectorizer(stop_words={'english'})
X = vectorizer.fit_transform(reviews)

true_k = 4
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=10)
model.fit(X)
labels=model.labels_
artcluster=pd.DataFrame(list(zip(tags,reviews,labels)),columns=['Tag','Review','cluster'])

cluster0 = artcluster[artcluster['cluster']== 0]
tag0 = cluster0['Tag']
review0 = cluster0['Review']
cluster1 = artcluster[artcluster['cluster']== 1]
tag1 = cluster1['Tag']
review1 = cluster1['Review']
cluster2 = artcluster[artcluster['cluster']== 2]
tag2 = cluster2['Tag']
review2 = cluster2['Review']
cluster3 = artcluster[artcluster['cluster']== 3]
tag3 = cluster3['Tag']
review3 = cluster3['Review']

In [32]:
labels = []
sent = []
for x in review0:
    sentiment_scores(x)
df0=pd.DataFrame(list(zip(tags,sent,labels)),columns=['Tag','Sentiment Score','Label'])
df0['Cluster'] = 0
labels = []
sent = []
for x in review1:
    sentiment_scores(x)
df1=pd.DataFrame(list(zip(tags,sent,labels)),columns=['Tag','Sentiment Score','Label'])
df1['Cluster'] = 1
labels = []
sent = []
for x in review2:
    sentiment_scores(x)
df2=pd.DataFrame(list(zip(tags,sent,labels)),columns=['Tag','Sentiment Score','Label'])
df2['Cluster'] = 2
labels = []
sent = []
for x in review3:
    sentiment_scores(x)
df3=pd.DataFrame(list(zip(tags,sent,labels)),columns=['Tag','Sentiment Score','Label'])
df3['Cluster'] = 3
full = df0.append(df1).append(df2).append(df3)
minmaxmean = full.groupby('Cluster').agg({'Sentiment Score': ['mean', 'min', 'max']})
minmaxmean.head()

Sentiment Score                
                   mean     min     max
Cluster                                
0              0.698964 -0.9897  0.9988
1              0.629051 -0.9078  0.9994
2              0.518512 -0.9501  0.9853
3              0.559377 -0.9136  0.9978

I found it interesting that the min and max for all the clusters were fairly similar. This means that the balancing of both the negative and positive reviews in the selection process was largely a success. However, it is interesting that even though the balancing of negative and positive reviews was successful, each cluster came out as overall positive on the mean. This is interesting to me that even though a review might have been a 1 star, there were still positive elements to some of them to allow this to happen.

### 3.	For extra credit, analyze sentiment of chunks as follows:
#### a.	Take the chunks from homework 5, and in Python, run each chunk individually through your sentiment analyzer that you used in question 1. If the chunk registers a nonneutral sentiment, save it in a tabular format (the chunk, the sentiment score).


In [65]:
# import chunks
chunks = pd.read_csv("C:/Users/nedei/Documents/School/NLP/Nathan_Deinlein_Homework_5_Allreviews.csv")
tags = chunks['0']
chunks = chunks.drop(columns = ["Unnamed: 0","0", "1"])
chunks.fillna('', inplace=True)
chunks = chunks.values.tolist()
N = ''
chunks = [[ele for ele in sub if ele != N] for sub in chunks]
import itertools
chunks = list(itertools.chain.from_iterable(chunks))

In [66]:
labels = []
sent = []
for x in chunks:
    sentiment_scores(x)


In [68]:
chunkdf = pd.DataFrame(list(zip(chunks,sent,labels)),columns=['Tag','Sentiment Score','Label'])
chunkdf = chunkdf[chunkdf['Label'] != 'Neutral']
chunkdf.head()

,Tag,Sentiment Score,Label
2,cinematic masterpiece,0.6249,Positive
3,'s worth,0.2263,Positive
5,wonderful films,0.5719,Positive
7,sure,0.3182,Positive
12,perfect film,0.5719,Positive


#### b.	Now sort the table twice, once to show the highest negative-sentiment-scoring chunks at the top and again to show the highest positive-sentiment-scoring chunks at the top. Examine the upper portions of both sorted lists, to identify any trends, and explain what you see. 
Submit all of your inputs and outputs and your code for this assignment, along with a brief written explanation of your findings. 


In [73]:
maxpos = chunkdf.sort_values(by=['Sentiment Score'], ascending = False)
maxpos.head(10)

,Tag,Sentiment Score,Label
580,great heart,0.8519,Positive
741,wonderful romance,0.8074,Positive
3028,great triumph,0.8020,Positive
3109,great kindness,0.7964,Positive
1206,great holiday film,0.7783,Positive
578,great support,0.7783,Positive
3261,wealth versus happiness,0.7783,Positive
988,beautiful friendship,0.7783,Positive
173,beautiful friendship,0.7783,Positive
1089,beautiful friendship,0.7783,Positive


In [74]:
minpos = chunkdf.sort_values(by=['Sentiment Score'])
minpos.head(10)

,Tag,Sentiment Score,Label
2925,brutal murders,-0.8442,Negative
2935,sinister gloom arrives,-0.8176,Negative
2572,murderous thieves,-0.8176,Negative
2983,crazy serial killers,-0.7717,Negative
2991,crazy serial killer,-0.7717,Negative
1722,stupid mess,-0.7096,Negative
2217,wicked witch,-0.7096,Negative
537,standard propaganda war film,-0.7096,Negative
2506,horrible mistake,-0.7096,Negative
2362,actual murder,-0.6908,Negative


I found it intereresting that the most positive sentiments were about friendships (Casablanca) and kindness and not about whether the movie was good or not. On the other end, all of the most negative things were about murder. 